In [ ]:
import numpy as np
from bnb.problem import OptimizationProblem
from matplotlib import pyplot as plt
import matplotlib
from scipy.optimize import minimize
import pandas as pd
import warnings
import seaborn as sns
from bnb.fml_solver import FMLSolver
sns.set()
matplotlib.rcParams.update({'font.size': 11, 'font.family': 'serif'})

In [ ]:
class GradientDescent(OptimizationProblem):
    
    def objective_function(self, p):
        return np.sum([
            segment.w * np.sum(p * segment.purchase_probabilities(p))
            for segment in self.segments
        ])
    
a = np.asarray([[-4], [6]])
b = np.asarray([0.01])
w_ = 0.9975
w = np.asarray([w_, 1 - w_])
problem = GradientDescent(a, b, w)

p = np.linspace(problem.p_lb, problem.p_ub, 100)
y = [problem.objective_function(np.asarray([p_])) for p_ in p]

plt.plot(p, y)
plt.title(w_)
plt.show()

In [ ]:
class GradientDescent(OptimizationProblem):
    
    def objective_function(self, p):
        return 1 / len(p) * np.sum([
            segment.w * np.sum(p * segment.purchase_probabilities(p))
            for segment in self.segments
        ])

    
for w_ in np.linspace(0.0, 0.1, 100)[:-1]:

    a = np.asarray([
        [ 7,  7,  7,  7, 7],
        [ 0,  0,  0,  0, 0]
    ])
    
    b = np.asarray([0.01, 0.01, 0.01, 0.01, 0.01])
    w = np.asarray([w_, 1 - w_])
    problem = GradientDescent(a, b, w)
    solutions = []
    failures = 0
    for _ in range(10):
        p_start = np.random.uniform(problem.p_lb, problem.p_ub)
        sol = minimize(lambda p: - problem.objective_function(p), p_start)
        if sol.success:
            solutions.append(- sol.fun * len(p_start))
        else:
            failures += 1
#     print("number of failures: ", failures)
    min_, max_ = np.min(solutions), np.max(solutions)
    if max_ / min_ > 1.01:
        print(w_)
        print(min_, max_)
        plt.hist(solutions)
        plt.show()


In [ ]:
class GradientDescent(OptimizationProblem):
    
    def objective_function(self, p):
        return 1 / len(p) * np.sum([
            segment.w * np.sum(p * segment.purchase_probabilities(p))
            for segment in self.segments
        ])

a = np.asarray([
    [ 6.5,  7],
    [-4.5, -4]
])

b = np.asarray([0.02, 0.01])
w_ = 0.002
w = np.asarray([w_, 1 - w_])
problem = GradientDescent(a, b, w)
trajectories = []
np.random.seed(5)

for i in range(10):
    
    def store_trajectory(p_):
        trajectory.append(np.copy(p_))
    
    p_start = np.random.uniform(problem.p_lb, problem.p_ub)
    trajectory = [p_start]
    sol = minimize(lambda p: - problem.objective_function(p), p_start, callback=store_trajectory)
    trajectories.append(trajectory)
        

In [ ]:
import matplotlib.cm as cm
import matplotlib.patches as patches


colors = cm.Greys(np.linspace(0.0, 1.0, len(trajectories)))

fig, ax = plt.subplots(figsize=(8, 6))

linspace = np.linspace(problem.p_lb * 0.8, problem.p_ub * np.asarray([1.6, 1.05]), 150)
p0, p1 = linspace[:, 0], linspace[:, 1]
P0, P1 = np.meshgrid(p0, p1)
Z = np.zeros(P0.shape)

for i in range(len(p0)):
    for j in range(len(p1)):
        p = np.asarray([P0[i, j], P1[i, j]])
        Z[i, j] = problem.objective_function(p)

cs = ax.contourf(P0, P1, Z, levels=50, cmap=plt.cm.jet, origin="lower")
# cs2 = ax.contour(cs, levels=cs.levels[::2], colors='r', origin="lower")
# ax.contour(P0, P1, Z, levels=50, color="grey")

for t, trajectory in enumerate(trajectories):

    for i in range(1, len(trajectory)):
    
        if i == 1:
            plt.plot(*trajectory[0], marker="d", markersize=10, color=colors[t])  #, fillstyle="none")
            
        if i == len(trajectory) - 1:
            plt.plot(*trajectory[-1], marker="*", markersize=10, color=colors[t])  #, fillstyle="none")
            
        
        arrowstyle = '<-' if 1 < i < len(trajectory) - 1 else '-'
        ax.annotate(
            "",
            xy=trajectory[i - 1],
            xytext=trajectory[i],
            arrowprops={
                'arrowstyle': arrowstyle, 'lw': 2, 'color': colors[t]
            },
            va='center', ha='center'
        )

ax.plot(*problem.segments[0].p_opt, marker="o", color="black")
ax.annotate(
    "Loyal segment optimal price",
    problem.segments[0].p_opt,
    problem.segments[0].p_opt + 60,
    va='center', ha='center', size=12,
                arrowprops={
                'arrowstyle': '-', 'lw': 1, 'color': "black"
            },
)

ax.plot(*problem.segments[1].p_opt, marker="o", color="black")
ax.annotate(
    "Bargain hunter segment optimal price",
    problem.segments[1].p_opt,
    problem.segments[1].p_opt - np.asarray([70, 70]),
    va='center', ha='center', size=12,
                arrowprops={
                'arrowstyle': '-', 'lw': 1, 'color': "black"
            },
)
# , problem.segments[1].p_opt

rect = patches.Rectangle(
    problem.p_lb, *(problem.p_ub - problem.p_lb),
    linewidth=1,edgecolor='black', facecolor='none', linestyle=":")



cbar = fig.colorbar(cs)
# Add the patch to the Axes
ax.add_patch(rect)

ax.set_xlabel(r"$p_1$", size=12)
ax.set_ylabel(r"$p_2$", size=12, rotation=0, labelpad=20)
plt.tight_layout()
plt.savefig("wrong_convergence.pdf")
plt.show()

In [ ]:
class GradientDescent(OptimizationProblem):
    
    def objective_function(self, p):
        return 1 / len(p) * np.sum([
            segment.w * np.sum(p * segment.purchase_probabilities(p))
            for segment in self.segments
        ])

    
for w_ in np.linspace(0.0001, 0.1, 100)[:-1]:

    a = np.asarray([
        [ 6.5,  7],
        [-4.5, -4]
    ])
    
    b = np.asarray([0.02, 0.01])
    w = np.asarray([w_, 1 - w_])
    problem = GradientDescent(a, b, w)
    solutions = []
    failures = 0
    runs = 10
    for _ in range(runs):
        p_start = np.random.uniform(problem.p_lb, problem.p_ub)
        sol = minimize(lambda p: - problem.objective_function(p), p_start)
        if sol.success:
            solutions.append(- sol.fun * len(p_start))
            # print(sol.x)
        else:
            failures += 1
    if failures > 0:
        print("Failures: ", failures / runs)
    min_, max_ = np.min(solutions), np.max(solutions)
    if max_ / min_ > 1.01:
        print(w_)
        print(min_, max_)
        plt.hist(solutions)
        plt.show()


In [ ]:
class GradientDescent(OptimizationProblem):
    
    def objective_function(self, p):
        return 1 / len(p) * np.sum([
            segment.w * np.sum(p * segment.purchase_probabilities(p))
            for segment in self.segments
        ])

    
for w_ in np.linspace(0.0, 1.0, 100)[:-1]:

    n_extra = 10

    a = np.asarray([
        [0, 0, 0, 7, 7] + [0] * n_extra,
        [7, 7, 7, 0, 0] + [0] * n_extra
    ])
    
    b = np.asarray([0.01, 0.001, 0.05, 0.075, 0.1] + [0.1] * n_extra)
    w = np.asarray([w_, 1 - w_])
    problem = GradientDescent(a, b, w)
    solutions = []
    failures = 0
    for _ in range(100):
        p_start = np.random.uniform(problem.p_lb, problem.p_ub)
        sol = minimize(
            lambda p: - problem.objective_function(p),
            p_start,
#             method="Nelder-Mead",
#             options={"maxfev": 10000}
        )
        if sol.success:
            solutions.append(- sol.fun * len(p_start))
        else:
            failures += 1
    print("number of failures: ", failures)
    min_, max_ = np.min(solutions), np.max(solutions)
    if max_ / min_ > 1.01:
        print(w_)
        print(min_, max_)
        plt.hist(solutions)
        plt.show()


In [ ]:
# class GradientDescent(OptimizationProblem):
    
#     def objective_function(self, p):
#         return np.sum([
#             segment.w * np.sum(p * segment.purchase_probabilities(p))
#             for segment in self.segments
#         ])

    

# while True:
    
#     w_ = np.random.uniform(0, 1)
#     a = np.asarray([
#         np.random.uniform(-4, 0, size=5),
#         np.random.uniform(7, 9, size=5),
#     ])
#     b = np.random.uniform(0.001, 0.1, size=10)
#     w = np.asarray([w_, 1 - w_])
#     problem = GradientDescent(a, b, w)
#     solutions = []
#     for _ in range(50):
#         p_start = np.random.uniform(problem.p_lb, problem.p_ub)
#         sol = minimize(
#             lambda p: - problem.objective_function(p),
#             p_start,
#             method="Nelder-Mead",
#             options={"maxfev": 10000}
#         )
#         if sol.success:
#             solutions.append(-sol.fun)
#     solutions = np.asarray(solutions)
#     min_, max_ = np.min(solutions), np.max(solutions)
#     fraction_wrong = np.mean(solutions < max_ * 0.9)
#     if fraction_wrong > 0.9:
#         raise Exception("found it")